In [1]:
import pathlib
import sys

import pandas as pd
import torch

sys.path.append("../helpers")
from transformer import (
    Decoder,
    DecoderBlock,
    Encoder,
    EncoderBlock,
    MultiHeadAttention,
    PositionWiseFeedForward,
    Transformer,
)

In [14]:
# load data
cls_file_path = pathlib.Path(
    "../../1.scDINO_analysis/1.scDINO_run/outputdir/mnist_photos/CLS_features/channel_binary_model_dino_deitsmall16_pretrain_full_checkpoint_features.csv"
).resolve(strict=True)

# load in the the data to a csv
cls_features = pd.read_csv(cls_file_path, header=None)
print(cls_features.shape)
cls_features.head()

(599600, 384)


,0,1,2,3,4,5,6,7,8,9,...,374,375,376,377,378,379,380,381,382,383
0,0.002911,-0.086649,0.024004,-0.052969,0.083575,0.050345,0.021312,-0.004625,0.046434,0.004440,...,0.032911,0.069589,0.015702,0.013550,0.018248,0.029374,0.088262,0.067843,-0.022405,0.012877
1,0.002911,-0.086649,0.024004,-0.052969,0.083575,0.050345,0.021312,-0.004625,0.046434,0.004440,...,0.032911,0.069589,0.015702,0.013550,0.018248,0.029374,0.088262,0.067843,-0.022405,0.012877
2,0.002911,-0.086649,0.024004,-0.052969,0.083575,0.050345,0.021312,-0.004625,0.046434,0.004440,...,0.032911,0.069589,0.015702,0.013550,0.018248,0.029374,0.088262,0.067843,-0.022405,0.012877
3,0.005962,-0.077774,0.009969,-0.064341,0.078557,0.048890,0.016025,0.001009,0.050749,-0.009278,...,0.039613,0.068184,0.022251,0.007466,0.026885,0.037929,0.074830,0.072795,-0.019385,0.002266
4,-0.007900,-0.079685,0.030329,-0.060220,0.062611,0.054394,0.020268,-0.002412,0.053426,0.020518,...,0.020964,0.069228,0.016879,0.007474,0.020761,0.025129,0.086611,0.065224,-0.027460,0.010036


In [15]:
# rename columns
cls_features.columns = [f"CLS_{i}" for i in range(cls_features.shape[1])]
cls_features.head()

,CLS_0,CLS_1,CLS_2,CLS_3,CLS_4,CLS_5,CLS_6,CLS_7,CLS_8,CLS_9,...,CLS_374,CLS_375,CLS_376,CLS_377,CLS_378,CLS_379,CLS_380,CLS_381,CLS_382,CLS_383
0,0.002911,-0.086649,0.024004,-0.052969,0.083575,0.050345,0.021312,-0.004625,0.046434,0.004440,...,0.032911,0.069589,0.015702,0.013550,0.018248,0.029374,0.088262,0.067843,-0.022405,0.012877
1,0.002911,-0.086649,0.024004,-0.052969,0.083575,0.050345,0.021312,-0.004625,0.046434,0.004440,...,0.032911,0.069589,0.015702,0.013550,0.018248,0.029374,0.088262,0.067843,-0.022405,0.012877
2,0.002911,-0.086649,0.024004,-0.052969,0.083575,0.050345,0.021312,-0.004625,0.046434,0.004440,...,0.032911,0.069589,0.015702,0.013550,0.018248,0.029374,0.088262,0.067843,-0.022405,0.012877
3,0.005962,-0.077774,0.009969,-0.064341,0.078557,0.048890,0.016025,0.001009,0.050749,-0.009278,...,0.039613,0.068184,0.022251,0.007466,0.026885,0.037929,0.074830,0.072795,-0.019385,0.002266
4,-0.007900,-0.079685,0.030329,-0.060220,0.062611,0.054394,0.020268,-0.002412,0.053426,0.020518,...,0.020964,0.069228,0.016879,0.007474,0.020761,0.025129,0.086611,0.065224,-0.027460,0.010036


In [16]:
transformer = Transformer(
    vocab_size=1000,
    model_dim=784,
    dropout=0.1,
    n_encoder_layers=6,
    n_decoder_layers=6,
    n_heads=8,
    batch_size=4,
    pad_idx=0,
)

 * vocab_size=1000
 * model_dim=784
 * dropout=0.1
 * n_encoder_layers=6
 * n_decoder_layers=6
 * n_heads=8
 * batch_size=4
 * pad_idx=0
